In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
from gurobipy import *
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
from rsome import cpt_solver as cpt
import pandas as pd
torch.manual_seed(42)
torch.cuda.manual_seed(42)

from Performance import performance_evaluation
perfs = performance_evaluation()

In [2]:
def Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump):
# #  ****** Coef generation *********
    from Data import data_generation
    data_gen = data_generation()
    # W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, coef_seed,data_generation_process) 
    # print("W_star = ",W_star[0,:])
    np.random.seed(coef_seed)
    x_test_all = {}; c_test_all = {}; x_train_all = {}; c_train_all = {}; W_star_all = {}; noise_train_all = {}; noise_test_all = {}
    for iter in iteration_all:
        DataPath_iter = DataPath +"iter="+str(iter)+"/"
        pathlib.Path(DataPath_iter).mkdir(parents=True, exist_ok=True)
        W_star = data_gen.generate_truth(DataPath_iter,lower, upper, p, d, iter,data_generation_process) 
        # #  ****** Data generation *********
        x_test_all[iter], c_test_all[iter], x_train_all[iter], c_train_all[iter], noise_train_all[iter],noise_test_all[iter],W_star_all[iter] = data_gen.generate_samples(iter,DataPath_iter,p, d, num_test, num_train, alpha, W_star, mis, num_test, 
                                data_generation_process, x_dist, e_dist, x_low, x_up, x_mean, x_var, bump) 
        # print()
    return x_test_all, c_test_all, x_train_all, c_train_all, noise_train_all,noise_test_all,W_star_all

In [3]:
def Implement_Oracle(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,iteration_all,num_feat,data_generation_process):
    cost_Oracle_with_noise_all = {}; cost_Oracle_wo_noise_all = {}
    for iter in iteration_all:
        if data_generation_process == "SPO_Data_Generation":
            cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T)/np.sqrt(num_feat) + 3
            cost_oracle_pred = (cost_oracle_ori ** mis + 1).T
            cost_Oracle_with_noise_all[iter] = perfs.compute_SPO_out_of_sample_Cost(arcs, grid,cost_oracle_pred,cost_oracle_pred,noise_test_all[iter])
            print("Oracle: iter=",iter,",cost_Oracle_with_noise_all=",np.nanmean(cost_Oracle_with_noise_all[iter]))

        if data_generation_process == "DDR_Data_Generation":
            cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T) + bump
            cost_oracle_pred = (cost_oracle_ori ** mis).T
            cost_Oracle_with_noise_all[iter] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_oracle_pred,cost_oracle_pred,noise_test_all[iter],True)
            cost_Oracle_wo_noise_all[iter] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_oracle_pred,cost_oracle_pred,noise_test_all[iter],False)

            # print("Oracle: iter=",iter,",cost_Oracle_with_noise_all=",np.nanmean(cost_Oracle_with_noise_all[iter]),",cost_Oracle_wo_noise_all=",np.nanmean(cost_Oracle_wo_noise_all[iter]))
    return cost_Oracle_with_noise_all,cost_Oracle_wo_noise_all

In [4]:
def Implement_OLS(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process):
    from OLS import ols_method
    ols_method_obj = ols_method()
    W_ols_all = {}; w0_ols_all = {}; t_ols_all = {}; obj_ols_all = {}
    cost_OLS_with_noise_all = {}; cost_OLS_wo_noise_all = {}
    for iter in iteration_all:
        # compute OLS performance
        W_ols_all[iter], w0_ols_all[iter], t_ols_all[iter], obj_ols_all[iter] = ols_method_obj.ols_solver("",x_train_all[iter], c_train_all[iter])
        cost_dem = (W_ols_all[iter] @ x_test_all[iter].T).T + w0_ols_all[iter]

        if data_generation_process == "SPO_Data_Generation":
            cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T)/np.sqrt(num_feat) + 3
            cost_oracle_pred = (cost_oracle_ori ** mis + 1).T
            cost_OLS_with_noise_all[iter] = perfs.compute_SPO_out_of_sample_Cost(arcs, grid,cost_dem,cost_oracle_pred,noise_test_all[iter])

        if data_generation_process == "DDR_Data_Generation":
            cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T) + bump
            cost_oracle_pred = (cost_oracle_ori ** mis).T
            cost_OLS_with_noise_all[iter] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_dem,cost_oracle_pred,noise_test_all[iter],True)
            cost_OLS_wo_noise_all[iter] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_dem,cost_oracle_pred,noise_test_all[iter],False)
        if iter % 20 == 0 and iter > 0:
            print("OLS: iter=",iter,",cost_OLS_with_noise_all =",np.nanmean(cost_OLS_with_noise_all[iter]))
    return cost_OLS_with_noise_all,cost_OLS_wo_noise_all

In [5]:
def Implement_DDR(mu_all,lamb_all,arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process):
    from DDR import DDR_method
    ddr_object = DDR_method()
    num_nodes = grid[0] * grid[0]

    w0_ddr_dict = {}; W_ddr_dict = {}
    cost_DDR_with_noise_all = {}; cost_DDR_wo_noise_all = {}
    for iter in iteration_all:
        for mu in mu_all:
            for lamb in lamb_all:
                w0_ddr_dict[iter,mu,lamb],W_ddr_dict[iter,mu,lamb],alpha_rst,obj_ddr = ddr_object.solve_DDR(arcs,lamb,mu,num_nodes,x_train_all[iter],c_train_all[iter])
                cost_pred = (W_ddr_dict[iter,mu,lamb] @ x_test_all[iter].T).T + w0_ddr_dict[iter,mu,lamb]

                if data_generation_process == "SPO_Data_Generation":
                    cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T)/np.sqrt(num_feat) + 3
                    cost_oracle_pred = (cost_oracle_ori ** mis + 1).T
                    cost_DDR_with_noise_all[iter,mu,lamb] = perfs.compute_SPO_out_of_sample_Cost(arcs, grid,cost_pred,cost_oracle_pred,noise_test_all[iter])

                if data_generation_process == "DDR_Data_Generation":
                    cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T) + bump
                    cost_oracle_pred = (cost_oracle_ori ** mis).T
                    # cost_DDR_with_noise_all[iter,mu,lamb] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_pred,cost_oracle_pred,noise_test_all[iter],True)
                    cost_DDR_wo_noise_all[iter,mu,lamb] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_pred,cost_oracle_pred,noise_test_all[iter],False)
        if iter % 1 == 0 and iter > 0:
            print("DDR: iter=",iter,",mu=",mu,",lamb=",lamb,",cost_DDR_with_noise_all =",np.nanmean(cost_DDR_with_noise_all[iter,mu,lamb]))

    return cost_DDR_with_noise_all,cost_DDR_wo_noise_all

# Parameters

In [6]:
grid = (5,5) # grid size
num_train = 100 # number of training data
num_feat = 5 # size of feature
num_test = 1000
deg = 1.0 # polynomial degree
e = 0.5 # scale of normal std or the range of uniform. For the error term

lower = 0 # coef lower bound
upper = 1 # coef upper bound
p = num_feat # num of features
d = (grid[0] - 1) * (grid[0] - 1) * 2 + 2 * (grid[0] - 1) # num of arcs
num_nodes = grid[0]*grid[0]
alpha = e # scale of normal std or the range of uniform. For the error term
mis = deg # model misspecification
coef_seed = 1

x_dist = 'uniform'
e_dist = 'normal'
x_low = -2
x_up = 2
x_mean = 2
x_var = 2
bump = 100

data_generation_process = "SPO_Data_Generation"
# data_generation_process = "DDR_Data_Generation"

current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)
DataPath = os.path.dirname(grandparent_directory) + '/Data/Shortest_Path_Reproduce/'+str(grid[0])+'by'+str(grid[1])+'_grid_' + data_generation_process + "_S=100/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
print("grandparent_directory:", grandparent_directory)
print("DataPath:", DataPath)
DataPath = DataPath + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_x_dist="+x_dist+"_num_test="+str(num_test)+"_diff_W/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)

grandparent_directory: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/5by5_grid_SPO_Data_Generation_S=100/


In [7]:
iteration_all = np.arange(0,100)
x_test_all, c_test_all, x_train_all, c_train_all,noise_train_all,noise_test_all,W_star_all = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)

In [8]:
from Network import network_design
Network = network_design()
arcs,arc_index_mapping = Network._getArcs(grid)

cost_Oracle_with_noise_all,cost_Oracle_wo_noise_all = Implement_Oracle(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,iteration_all,num_feat,data_generation_process)

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13
Oracle: iter= 0 ,cost_Oracle_with_noise_all= 29.3866090601035
Oracle: iter= 1 ,cost_Oracle_with_noise_all= 29.206888258544698
Oracle: iter= 2 ,cost_Oracle_with_noise_all= 29.84722265331638
Oracle: iter= 3 ,cost_Oracle_with_noise_all= 29.54973437815811
Oracle: iter= 4 ,cost_Oracle_with_noise_all= 30.04309887526118
Oracle: iter= 5 ,cost_Oracle_with_noise_all= 29.822800718946777
Oracle: iter= 6 ,cost_Oracle_with_noise_all= 29.553496105402882
Oracle: iter= 7 ,cost_Oracle_with_noise_all= 29.344619889515098
Oracle: iter= 8 ,cost_Oracle_with_noise_all= 29.18228176386138
Oracle: iter= 9 ,cost_Oracle_with_noise_all= 29.529635487694396
Oracle: iter= 10 ,cost_Oracle_with_noise_all= 29.314580959557848
Oracle: iter= 11 ,cost_Oracle_with_noise_all= 29.861832481945385
Oracle: iter= 12 ,cost_Oracle_with_noise_all= 29.60493782617165
Oracle: iter= 13 ,cost_Oracle_with_noise_all= 29.38400175647671
Oracle: iter= 14 

In [9]:
cost_OLS_with_noise_all,cost_OLS_wo_noise_all = Implement_OLS(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process)

OLS: iter= 20 ,cost_OLS_with_noise_all = 29.753776569887517
OLS: iter= 40 ,cost_OLS_with_noise_all = 30.269029629341485
OLS: iter= 60 ,cost_OLS_with_noise_all = 29.890047525489848
OLS: iter= 80 ,cost_OLS_with_noise_all = 29.776666480830645


In [10]:
mu_all = np.round(np.arange(0.1,1.0,0.1),4)
lamb_all = np.round(np.arange(0.0,1.0,0.1),4)
cost_DDR_with_noise_all,cost_DDR_wo_noise_all = Implement_DDR(mu_all,lamb_all,arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process)

DDR: iter= 1 ,mu= 0.9 ,lamb= 0.9 ,cost_DDR_with_noise_all = 29.65109719021676
DDR: iter= 2 ,mu= 0.9 ,lamb= 0.9 ,cost_DDR_with_noise_all = 30.356211891837965
DDR: iter= 3 ,mu= 0.9 ,lamb= 0.9 ,cost_DDR_with_noise_all = 29.788252922121124
DDR: iter= 4 ,mu= 0.9 ,lamb= 0.9 ,cost_DDR_with_noise_all = 30.36010856426727
DDR: iter= 5 ,mu= 0.9 ,lamb= 0.9 ,cost_DDR_with_noise_all = 30.169854805130687
DDR: iter= 6 ,mu= 0.9 ,lamb= 0.9 ,cost_DDR_with_noise_all = 29.937245822291704
DDR: iter= 7 ,mu= 0.9 ,lamb= 0.9 ,cost_DDR_with_noise_all = 29.72658514639245
DDR: iter= 8 ,mu= 0.9 ,lamb= 0.9 ,cost_DDR_with_noise_all = 29.480551341650383
DDR: iter= 9 ,mu= 0.9 ,lamb= 0.9 ,cost_DDR_with_noise_all = 29.87084667377576
DDR: iter= 10 ,mu= 0.9 ,lamb= 0.9 ,cost_DDR_with_noise_all = 29.67115809059136
DDR: iter= 11 ,mu= 0.9 ,lamb= 0.9 ,cost_DDR_with_noise_all = 30.231641970699158
DDR: iter= 12 ,mu= 0.9 ,lamb= 0.9 ,cost_DDR_with_noise_all = 29.863476901813033
DDR: iter= 13 ,mu= 0.9 ,lamb= 0.9 ,cost_DDR_with_noise

In [11]:
with open(DataPath+'cost_OLS_with_noise_all.pkl', "wb") as tf:
    pickle.dump(cost_OLS_with_noise_all,tf)
with open(DataPath+'cost_Oracle_with_noise_all.pkl', "wb") as tf:
    pickle.dump(cost_Oracle_with_noise_all,tf)
with open(DataPath+'cost_DDR_with_noise_all.pkl', "wb") as tf:
    pickle.dump(cost_DDR_with_noise_all,tf)

In [12]:
with open(DataPath+'cost_OLS_with_noise_all.pkl', "rb") as tf:
    cost_OLS_with_noise_all = pickle.load(tf)
with open(DataPath+'cost_Oracle_with_noise_all.pkl', "rb") as tf:
    cost_Oracle_with_noise_all = pickle.load(tf)
with open(DataPath+'cost_DDR_with_noise_all.pkl', "rb") as tf:
    cost_DDR_with_noise_all = pickle.load(tf)

### Comparison

In [13]:
def cross_compare2plus(c_item, c_base, c_oracle):
    N = len(c_item)
    c_diff = c_base - c_item
    lbel = np.zeros((N,1))
    
    equals = np.sum(c_diff == 0)
    wins = np.sum(c_diff > 0) # indicate num of c_item is lower than c_base
    lose = np.sum(c_diff < 0)
    
    lbel[c_diff < 0] = 1
    lbel[c_diff > 0] = -1
    
#     print(N, equals, wins, lose)
    if N == equals:
        win_ratio = 0.5
    else:
        win_ratio = wins/(N - equals)
    cost_reduction = (np.mean(c_diff))/np.abs(np.mean(c_base))
    regret_reduction = (np.mean(c_diff))/np.abs(np.mean(c_base) - np.mean(c_oracle))
    return win_ratio, cost_reduction, regret_reduction

In [14]:
h2h_ddr_vs_ols_all = {}; cost_reduction_ddr_vs_ols_all = {}; regret_reduction_ddr_vs_ols_all = {}
h2h_ddr_vs_ols_avg = np.zeros((len(mu_all),len(lamb_all))); regret_ddr_vs_ols_avg = np.zeros((len(mu_all),len(lamb_all)))

mu_index = 0
for mu in mu_all:
    lamb_index = 0
    for lamb in lamb_all:

        h2h_ddr_ols = np.zeros(len(iteration_all)); cost_reduction_ddr_vs_ols = np.zeros(len(iteration_all)); regret_reduction_ddr_vs_ols = np.zeros(len(iteration_all))
        for iter_index in range(len(iteration_all)):
            iter = iteration_all[iter_index]
            h2h_ddr_ols[iter_index],cost_reduction_ddr_vs_ols[iter_index],regret_reduction_ddr_vs_ols[iter_index] = cross_compare2plus(cost_DDR_with_noise_all[iter,mu,lamb], cost_OLS_with_noise_all[iter], cost_Oracle_with_noise_all[iter])
        h2h_ddr_vs_ols_avg[mu_index,lamb_index] = np.nanmean(h2h_ddr_ols)
        regret_ddr_vs_ols_avg[mu_index,lamb_index] = np.nanmean(regret_reduction_ddr_vs_ols)
        print("mu=",mu,",lamb=",lamb,"h2h_ddr_ols=",np.nanmean(h2h_ddr_ols),"regret=",np.nanmean(regret_reduction_ddr_vs_ols))
        lamb_index = lamb_index + 1
    
        h2h_ddr_vs_ols_all[mu,lamb] = h2h_ddr_ols
        cost_reduction_ddr_vs_ols_all[mu,lamb] = cost_reduction_ddr_vs_ols
        regret_reduction_ddr_vs_ols_all[mu,lamb] = regret_reduction_ddr_vs_ols
    mu_index = mu_index + 1

mu= 0.1 ,lamb= 0.0 h2h_ddr_ols= 0.5 regret= 0.0
mu= 0.1 ,lamb= 0.1 h2h_ddr_ols= 0.5054385181142962 regret= 0.0008182865055642739
mu= 0.1 ,lamb= 0.2 h2h_ddr_ols= 0.5019022219578873 regret= 0.00024680200489297054
mu= 0.1 ,lamb= 0.3 h2h_ddr_ols= 0.4939105547446935 regret= -0.0054629493523739294
mu= 0.1 ,lamb= 0.4 h2h_ddr_ols= 0.48554207547175493 regret= -0.013951982938941317
mu= 0.1 ,lamb= 0.5 h2h_ddr_ols= 0.47422402629487387 regret= -0.027772911598010076
mu= 0.1 ,lamb= 0.6 h2h_ddr_ols= 0.46539300213764084 regret= -0.043678841405654624
mu= 0.1 ,lamb= 0.7 h2h_ddr_ols= 0.4520284409338725 regret= -0.06441425029184952
mu= 0.1 ,lamb= 0.8 h2h_ddr_ols= 0.4386004413782556 regret= -0.09216405326802832
mu= 0.1 ,lamb= 0.9 h2h_ddr_ols= 0.42803609469724835 regret= -0.11878315345231873
mu= 0.2 ,lamb= 0.0 h2h_ddr_ols= 0.5 regret= 0.0
mu= 0.2 ,lamb= 0.1 h2h_ddr_ols= 0.504197363091746 regret= 0.00013077908938838846
mu= 0.2 ,lamb= 0.2 h2h_ddr_ols= 0.5027239289731302 regret= 0.0007695041988238504
mu= 0.2 ,l

In [15]:
regret_DDR_vs_OLS_para_avg_df = pd.DataFrame(regret_ddr_vs_ols_avg)
regret_DDR_vs_OLS_para_avg_df.index = ["$\mu="+str(mu)+"$" for mu in mu_all]
regret_DDR_vs_OLS_para_avg_df.columns = ["$\lambda="+str(lamb)+"$" for lamb in lamb_all]
regret_DDR_vs_OLS_para_avg_df.to_csv(DataPath+"regret_ddr_vs_ols_avg.csv")

In [16]:
regret_DDR_vs_OLS_para_avg_df

,$\lambda=0.0$,$\lambda=0.1$,$\lambda=0.2$,$\lambda=0.3$,$\lambda=0.4$,$\lambda=0.5$,$\lambda=0.6$,$\lambda=0.7$,$\lambda=0.8$,$\lambda=0.9$
$\mu=0.1$,0.0,0.000818,0.000247,-0.005463,-0.013952,-0.027773,-0.043679,-0.064414,-0.092164,-0.118783
$\mu=0.2$,0.0,0.000131,0.000770,-0.003586,-0.006339,-0.014306,-0.025330,-0.038919,-0.053644,-0.071788
$\mu=0.3$,0.0,0.000316,0.002166,0.002955,-0.000761,-0.006703,-0.012069,-0.023053,-0.033479,-0.046584
$\mu=0.4$,0.0,0.000691,0.000412,0.000723,-0.002279,-0.004620,-0.010429,-0.016995,-0.024071,-0.031709
$\mu=0.5$,0.0,-0.000301,0.000650,0.000872,0.000029,-0.002163,-0.006567,-0.009910,-0.014280,-0.021110
$\mu=0.6$,0.0,-0.000267,-0.000252,-0.000464,-0.000738,-0.001217,-0.002906,-0.004713,-0.008421,-0.011707
$\mu=0.7$,0.0,-0.000358,-0.000004,-0.000350,-0.000540,-0.000062,-0.000819,-0.001700,-0.003462,-0.003704
$\mu=0.8$,0.0,0.000002,-0.000106,0.000562,0.000217,0.000691,0.000636,0.000656,0.000095,-0.000699
$\mu=0.9$,0.0,0.000424,-0.000289,-0.000195,-0.000448,-0.000409,-0.000024,0.000017,-0.000331,-0.000160


In [17]:
h2h_DDR_vs_OLS_para_avg_df = pd.DataFrame(h2h_ddr_vs_ols_avg * 100)
h2h_DDR_vs_OLS_para_avg_df.index = ["$\mu="+str(mu)+"$" for mu in mu_all]
h2h_DDR_vs_OLS_para_avg_df.columns = ["$\lambda="+str(lamb)+"$" for lamb in lamb_all]
h2h_DDR_vs_OLS_para_avg_df.to_csv(DataPath+"h2h_ddr_vs_ols_avg.csv")

In [18]:
h2h_DDR_vs_OLS_para_avg_df

,$\lambda=0.0$,$\lambda=0.1$,$\lambda=0.2$,$\lambda=0.3$,$\lambda=0.4$,$\lambda=0.5$,$\lambda=0.6$,$\lambda=0.7$,$\lambda=0.8$,$\lambda=0.9$
$\mu=0.1$,50.0,50.543852,50.190222,49.391055,48.554208,47.422403,46.539300,45.202844,43.860044,42.803609
$\mu=0.2$,50.0,50.419736,50.272393,49.348802,48.933186,48.369444,47.520337,46.806229,46.035184,45.273431
$\mu=0.3$,50.0,50.913978,50.942576,50.798585,50.129227,49.127494,48.841545,48.039523,47.226476,46.364369
$\mu=0.4$,50.0,52.171327,51.002916,50.867792,49.859536,49.532259,48.762078,48.192180,47.527694,47.076940
$\mu=0.5$,50.0,51.258936,51.460817,51.180108,50.675516,50.057719,49.199146,49.222032,48.597400,47.750001
$\mu=0.6$,50.0,50.787486,50.975030,50.959475,50.645630,50.317932,49.793824,49.508387,48.800142,48.395647
$\mu=0.7$,50.0,50.301229,52.130152,50.813531,50.570216,51.081295,50.697381,50.144722,49.802546,49.710205
$\mu=0.8$,50.0,51.827201,51.356050,52.255970,52.302122,51.731718,51.559921,51.616919,51.424037,50.938521
$\mu=0.9$,50.0,50.461905,50.190873,52.126825,50.181132,50.407028,50.548354,51.262953,50.799692,50.592271


### H2H_regret

In [19]:
from Figures import regret_and_h2h_figure
rhf = regret_and_h2h_figure()

In [20]:
# mu = 0.7
# lamb = 0.1
# all_x = h2h_ddr_vs_ols_all[mu,lamb] * 100
# all_y = regret_reduction_ddr_vs_ols_all[mu,lamb] * 100
# figure_name = DataPath + "h2h_regret_mu"+str(mu)+"_lamb="+str(lamb)+"_"
# rhf.figure_plot_upright(all_x,all_y,figure_name)

# EPO(SPO+,...)

In [ ]:
def Implement_EPO(DataPath,iteration_all,batch_size,num_epochs,method_names,W_star_all,bump,x_train_all,c_train_all,x_test_all,noise_test_all,\
                  arcs,grid,epo_runner,perfs):
    W_EPO_all = {}; w0_EPO_all = {}
    cost_EPO_all = {}
    for iter in iteration_all:
        DataPath_seed = DataPath +"iter="+str(iter)+"/"
        pathlib.Path(DataPath_seed).mkdir(parents=True, exist_ok=True)
        # print("*** seed = ",seed,": Run EPO ******")
        W_EPO_all[iter],w0_EPO_all[iter] = epo_runner.run(method_names,DataPath_seed,batch_size,num_feat,grid,num_epochs,\
                                        x_train_all[iter],c_train_all[iter],arcs)
        
        cost_dem = (W_EPO_all[iter] @ x_test_all[iter].T).T + w0_EPO_all[iter]
        if data_generation_process == "SPO_Data_Generation":
            cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T)/np.sqrt(num_feat) + 3
            cost_oracle_pred = (cost_oracle_ori ** mis + 1).T
            cost_EPO_all[iter] = perfs.compute_SPO_out_of_sample_Cost(arcs, grid,cost_dem,cost_oracle_pred,noise_test_all[iter])

        if data_generation_process == "DDR_Data_Generation":
            cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T) + bump
            cost_oracle_pred = (cost_oracle_ori ** mis).T
            cost_EPO_all[iter] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_dem,cost_oracle_pred,noise_test_all[iter])
        print(method_names[0],": iter=",iter,",cost=",np.nanmean(cost_EPO_all[iter]))

    return W_EPO_all,w0_EPO_all,cost_EPO_all

In [22]:
# data_generation_process = "SPO_Data_Generation"
# # data_generation_process = "DDR_Data_Generation"

# grid_all = [(2,2),(3,3),(4,4),(5,5)] # grid size
# max_regret_arr = []
# for grid in grid_all:
#     d = (grid[0] - 1) * (grid[0] - 1) * 2 + 2 * (grid[0] - 1) # num of arcs

#     current_directory = os.getcwd()
#     parent_directory = os.path.dirname(current_directory)
#     grandparent_directory = os.path.dirname(parent_directory)
#     DataPath_parents = os.path.dirname(grandparent_directory) + '/Data/Shortest_Path_Reproduce/'+str(grid[0])+'by'+str(grid[1])+'_grid_' + data_generation_process + "/"
#     print("DataPath_parents:", DataPath_parents)
#     DataPath_current = DataPath_parents + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_x_dist="+x_dist+"_num_test="+str(num_test)+"_diff_W/"
#     print("DataPath_current:", DataPath_current)
#     regret_ddr_vs_ols_avg = pd.read_csv(DataPath_current +'regret_ddr_vs_ols_avg.csv',index_col=0)
#     regret_np = pd.DataFrame(regret_ddr_vs_ols_avg.values).applymap(lambda x: float(x.strip('%')) / 100).values
#     print("max regert = ",np.max(regret_np)*100)
#     max_regret_arr.append(np.max(regret_np)*100)
# max_regret_arr = np.asarray(max_regret_arr)